## Install dependencies

In [ ]:
!pip install beautifulsoup4==4.12.3 grobid-client-python==0.0.8 lxml==5.3.0

## Extract from the sample PDF

In [ ]:
import multiprocessing
import os

from bs4 import BeautifulSoup
from grobid_client.grobid_client import GrobidClient

Before creating the client, make sure that the GROBID server is up and running:

```sh
docker compose up grobid
```

In [ ]:
grobid_server = os.environ.get("GROBID_SERVICE_URL", "http:localhost:8070")
n = 2 * multiprocessing.cpu_count()  # Assumes hyperthreading
output = "../data/grobid"
pdfs = "../aicacia/extraction/example/pdf"

client = GrobidClient(grobid_server=grobid_server)

In [ ]:
client.process("processFulltextDocument", pdfs, output=output, n=n)

## Parsing the TEI output with Beautiful Soup

In [ ]:
with open("../data/grobid/sample.grobid.tei.xml") as f:
    soup = BeautifulSoup(f, "lxml-xml")

In [ ]:
title_stmt = soup.find("titleStmt")

In [ ]:
title_stmt.title.text

'Bi-criteria Algorithm for Scheduling Jobs on Cluster Platforms'